# Pulse Sequence Evaluation

Will Kaufman, 2020

This notebook verifies the fidelity of pulse sequences identified by a reinforcement learning algorithm (as opposed to traditional techniques like average Hamiltonian theory).

In [ ]:
import spinSimulation as ss
import rlPulse as rlp
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px

In [ ]:
import importlib
importlib.reload(ss)
importlib.reload(rlp)

# Initialize spin system

This sets the parameters of the system ($N$ spin-1/2 particles, which corresponds to a Hilbert space with dimension $2^N$). For the purposes of simulation, $\hbar \equiv 1$.

The total internal Hamiltonian is given by
$$
H_\text{int} = C H_\text{dip} + \Delta \sum_i^N I_z^{(i)}
$$
where $C$ is the coupling strength, $\Delta$ is the chemical shift strength (each spin is assumed to be identical), and $H_\text{dip}$ is given by
$$
H_\text{dip} = \sum_{i,j}^N d_{i,j} \left(3I_z^{(i)}I_z^{(j)} - \mathbf{I}^{(i)} \cdot \mathbf{I}^{(j)}\right)
$$

The WAHUHA pulse sequence is designed to remove the dipolar interaction term from the internal Hamiltonian. The pulse sequence is $\tau, P_{-x}, \tau, P_{y}, \tau, \tau, P_{-y}, \tau, P_{x}, \tau$.
The zeroth-order average Hamiltonian for the WAHUHA pulse sequence is
$$
H_\text{WHH} = \Delta / 3 \sum_i^N I_x^{(i)} + I_y^{(i)} + I_z^{(i)}
$$

In [ ]:
N = 4
dim = 2**N
coupling = 2*np.pi * 5e3    # coupling strength
delta = 2*np.pi * 500       # chemical shift strength (for identical spins)

(x,y,z) = (ss.x, ss.y, ss.z)
(X,Y,Z) = ss.getTotalSpin(N, dim)

Ux = ss.getRotation(X, np.pi/2)
Uxbar = ss.getRotation(X, -np.pi/2)
Uy = ss.getRotation(Y, np.pi/2)
Uybar = ss.getRotation(Y, -np.pi/2)

# Evaluate pulse sequences

Canditate pulse sequences:

Number | Max reward | Pulse sequence | Date
---|---|---|---
1| 4.90 | tau, xbar, tau, ybar, tau, x y | (6/6)
2| 7.12 | tau, xbar y, tau, xbar, tau, ybar xbar^2 | (6/6)
3| 7.30 | xbar, tau, xbar, tau, ybar x | (6/6)
4| 7.31 | x ybar xbar, tau, x ybar^2, tau, y, tau, xbar y^2 x^2 | (6/9)

I'll go in order and evaluate each one independently

## 1

tau, xbar, tau, ybar, tau, x y

In [ ]:
# TODO define the propagators in code here...
# run for lots of random dipolar interactions

fidelities = []
delays = []
pulseWidths = []

for delay in np.geomspace(1e-6,10e-6,5):
    for pulseWidth in np.geomspace(.01e-6,1e-6,5):
        for i in range(100):
            Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
            HWHH0 = ss.getHWHH0(X,Y,Z,delta)
            UWHH0 = ss.getPropagator(HWHH0, 3*delay)
            
            if pulseWidth > 0:
                Ux = ss.getPropagator(Hint + X*np.pi/2/pulseWidth, pulseWidth)
                Uy = ss.getPropagator(Hint + Y*np.pi/2/pulseWidth, pulseWidth)
                Uxbar = ss.getPropagator(Hint - X*np.pi/2/pulseWidth, pulseWidth)
                Uybar = ss.getPropagator(Hint - Y*np.pi/2/pulseWidth, pulseWidth)
            Utau = ss.getPropagator(Hint, delay)

            Useq = Uy @ Ux @ Utau @ Uybar @ Utau @ Uxbar @ Utau
            fidelities.append(ss.fidelity(Useq, UWHH0))
            delays.append(delay)
            pulseWidths.append(pulseWidth)

In [ ]:
# plot results
d = {'fidelity': fidelities, 'delay': delays, 'pulseWidth': pulseWidths}
df1 = pd.DataFrame(data=d)
fig1 = px.scatter_3d(df1, x='delay', y='pulseWidth', z='fidelity', opacity=0.7)
fig1.update_layout(scene={'xaxis': {'type': 'log'}, 'yaxis': {'type': 'log'}})
fig1.show()

## 2

tau, xbar y, tau, xbar, tau, ybar xbar^2

In [ ]:
# TODO define the propagators in code here...
# run for lots of random dipolar interactions

fidelities = []
delays = []
pulseWidths = []
types = []
times = []
reps = []

for delay in np.geomspace(1e-6,10e-6,4):
    for pulseWidth in np.geomspace(.01e-6,1e-6,4):
        for i in range(250):
            Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
            HWHH0 = ss.getHWHH0(X,Y,Z,delta)
            UWHH0 = ss.getPropagator(HWHH0, 3*delay)
            
            if pulseWidth > 0:
                Ux = ss.getPropagator(Hint + X*np.pi/2/pulseWidth, pulseWidth)
                Uy = ss.getPropagator(Hint + Y*np.pi/2/pulseWidth, pulseWidth)
                Uxbar = ss.getPropagator(Hint - X*np.pi/2/pulseWidth, pulseWidth)
                Uybar = ss.getPropagator(Hint - Y*np.pi/2/pulseWidth, pulseWidth)
            Utau = ss.getPropagator(Hint, delay)

            Useq = Uxbar @ Uxbar @ Uybar @ Utau @ Uxbar @ Utau @ Uxbar @ Uybar @ Utau
            tseq = 6*pulseWidth + 3*delay
            UWHH = Utau @ Ux @ Utau @ Uybar @ Utau @ Utau @ Uy @ Utau @ Uxbar @ Utau
            tWHH = 4*pulseWidth + 6*delay
            fidelities.append(ss.fidelity(Useq**np.ceil(1e-3/tseq), UWHH0))
            delays.append(delay)
            pulseWidths.append(pulseWidth)
            types.append('Candidate')
            times.append(tseq * np.ceil(1e-3/tseq))
            reps.append(np.ceil(1e-3/tseq))
            # and record WAHUHA sequence benchmark
            fidelities.append(ss.fidelity(UWHH**np.ceil(1e-3/tWHH), UWHH0))
            delays.append(delay)
            pulseWidths.append(pulseWidth)
            types.append('WHH')      
            times.append(tWHH * np.ceil(1e-3/tWHH))
            reps.append(np.ceil(1e-3/tWHH))

In [ ]:
d = {'fidelity': fidelities, 'delay': delays, \
     'pulseWidth': pulseWidths, 'type': types, 'time': times, \
     'rep': reps}
df2 = pd.DataFrame(data=d)
df2.to_csv("candidate2.csv")

## 3

xbar, tau, xbar, tau, ybar x

In [ ]:
# TODO define the propagators in code here...
# run for lots of random dipolar interactions

fidelities = []
delays = []
pulseWidths = []
types = []
times = []
reps = []

for delay in np.geomspace(1e-6,10e-6,4):
    for pulseWidth in np.geomspace(.01e-6,1e-6,4):
        for i in range(250):
            Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
            HWHH0 = ss.getHWHH0(X,Y,Z,delta)
            UWHH0 = ss.getPropagator(HWHH0, 3*delay)
            
            if pulseWidth > 0:
                Ux = ss.getPropagator(Hint + X*np.pi/2/pulseWidth, pulseWidth)
                Uy = ss.getPropagator(Hint + Y*np.pi/2/pulseWidth, pulseWidth)
                Uxbar = ss.getPropagator(Hint - X*np.pi/2/pulseWidth, pulseWidth)
                Uybar = ss.getPropagator(Hint - Y*np.pi/2/pulseWidth, pulseWidth)
            Utau = ss.getPropagator(Hint, delay)

            Useq = Utau @ Ux @ Uybar @ Utau @ Uy @ Utau @ Uxbar
            tseq = 6*pulseWidth + 3*delay
            UWHH = Utau @ Ux @ Utau @ Uybar @ Utau @ Utau @ Uy @ Utau @ Uxbar @ Utau
            tWHH = 4*pulseWidth + 6*delay
            fidelities.append(ss.fidelity(Useq**np.ceil(1e-3/tseq), UWHH0))
            delays.append(delay)
            pulseWidths.append(pulseWidth)
            types.append('Candidate')
            times.append(tseq * np.ceil(1e-3/tseq))
            reps.append(np.ceil(1e-3/tseq))
            # and record WAHUHA sequence benchmark
            fidelities.append(ss.fidelity(UWHH**np.ceil(1e-3/tWHH), UWHH0))
            delays.append(delay)
            pulseWidths.append(pulseWidth)
            types.append('WHH')      
            times.append(tWHH * np.ceil(1e-3/tWHH))
            reps.append(np.ceil(1e-3/tWHH))

In [ ]:
d = {'fidelity': fidelities, 'delay': delays, \
     'pulseWidth': pulseWidths, 'type': types, 'time': times, \
     'rep': reps}
df3 = pd.DataFrame(data=d)
df3.to_csv("candidate3.csv")

## 4

x ybar xbar, tau, x ybar^2, tau, y, tau, xbar y^2 x^2

In [ ]:
# TODO define the propagators in code here...
# run for lots of random dipolar interactions

fidelities = []
delays = []
pulseWidths = []
types = []
times = []
reps = []

for delay in np.geomspace(1e-6,10e-6,4):
    for pulseWidth in np.geomspace(.01e-6,1e-6,4):
        for i in range(250):
            Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
            HWHH0 = ss.getHWHH0(X,Y,Z,delta)
            UWHH0 = ss.getPropagator(HWHH0, 3*delay)
            
            if pulseWidth > 0:
                Ux = ss.getPropagator(Hint + X*np.pi/2/pulseWidth, pulseWidth)
                Uy = ss.getPropagator(Hint + Y*np.pi/2/pulseWidth, pulseWidth)
                Uxbar = ss.getPropagator(Hint - X*np.pi/2/pulseWidth, pulseWidth)
                Uybar = ss.getPropagator(Hint - Y*np.pi/2/pulseWidth, pulseWidth)
            Utau = ss.getPropagator(Hint, delay)

            Useq = Ux @ Ux @ Uy @ Uy @ Uxbar @ Utau @ Uy @ Utau @ Uybar @ Uybar @ Ux @ Utau @ Uxbar @ Uybar @ Ux
            tseq = 12*pulseWidth + 3*delay
            UWHH = Utau @ Ux @ Utau @ Uybar @ Utau @ Utau @ Uy @ Utau @ Uxbar @ Utau
            tWHH = 4*pulseWidth + 6*delay
            fidelities.append(ss.fidelity(Useq**np.ceil(1e-3/tseq), UWHH0))
            delays.append(delay)
            pulseWidths.append(pulseWidth)
            types.append('Candidate')
            times.append(tseq * np.ceil(1e-3/tseq))
            reps.append(np.ceil(1e-3/tseq))
            # and record WAHUHA sequence benchmark
            fidelities.append(ss.fidelity(UWHH**np.ceil(1e-3/tWHH), UWHH0))
            delays.append(delay)
            pulseWidths.append(pulseWidth)
            types.append('WHH')      
            times.append(tWHH * np.ceil(1e-3/tWHH))
            reps.append(np.ceil(1e-3/tWHH))

In [ ]:
d = {'fidelity': fidelities, 'delay': delays, \
     'pulseWidth': pulseWidths, 'type': types, 'time': times, \
     'rep': reps}
df4 = pd.DataFrame(data=d)

In [ ]:
df4.to_csv("candidate4.csv")

In [ ]:
# plot results
fig4 = px.scatter_3d(df4, x='delay', y='pulseWidth', z='fidelity', opacity=0.7)
fig4.update_layout(scene={'xaxis': {'type': 'log'}, 'yaxis': {'type': 'log'}})
fig4.show()